In [1]:
# Import neccesary libraries

import pandas as pd
import seaborn as sns
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os
import cv2

2024-04-20 15:02:21.021934: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Training images preprocessing
def preprocess_images(data_dir, img_size):
    x_train = []
    y_train = []
    
    # Loop through all image folders in training folder
    for folder in os.listdir(data_dir):
        label = folder
        if label == '.DS_Store':
            continue
            
        folder_path = os.path.join(data_dir, folder)
        #print(folder_path)
        
        # Loop through all images in each classification folder
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path)
            
            # If not an image: skip
            if img is None:
                continue
                
            img = cv2.resize(img, (img_size, img_size))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            x_train.append(img)
            y_train.append(label)
            
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    return x_train, y_train

In [17]:
# Training images file path
data_dir = './images/training'
data_dir = pathlib.Path(data_dir)

# Number of images
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

# Preprocess training images with specific image size
img_size = 224
x_train, y_train = preprocess_images(data_dir, img_size)

974
